In [106]:
import pandas as pd
import numpy as np
import time
import os
import shutil
import re

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Crawler Description

This crawler downloads Uniswap v3 swap data from Flipside Crypto.

Enter the pool address that you want to download for manually below.

In [107]:
download_path = "/Users/zhicong/Desktop/KCL/Ethereum_DEX_Data/uniswapv3_swaps"  # path to store PDFs
pool_address = "0x840deeef2f115cf50da625f7368c24af6fe74410"
pool_name = "cbETH-WETH 500 10"

os.chdir(download_path)
os.mkdir(pool_name)

Initiate Selenium.

In [108]:
chromeOptions = webdriver.ChromeOptions()
#Get through DDos Protection by Cloudflare
chromeOptions.add_argument("--disable-blink-features=AutomationControlled")
#Set download paths
prefs = {"download.default_directory": download_path}
chromeOptions.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(service = ChromeService(ChromeDriverManager().install()), options = chromeOptions)

Open Flipside Crypto and login.

In [109]:
driver.get("https://flipsidecrypto.xyz/")
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[3]/div[1]/div[1]/div/nav/div[9]/div/button[2]"))).click()
time.sleep(np.random.randint(5, 10))

#Enter username and password
driver.find_element(By.XPATH, "//*[@id='username']").send_keys("zhiconghu@gmail.com")
driver.find_element(By.XPATH, "//*[@id='password']").send_keys("Zhicong12345")
driver.find_element(By.XPATH, "/html/body/div/main/section/div/div/div/form/div[3]").click()
time.sleep(np.random.randint(10, 15))

Open the query named "Temp".

In [110]:
#Find the query with name "Temp"
query_collections = driver.find_elements(By.XPATH, "//a/div[@class='flex items-center justify-between w-full']")
for i in query_collections:
    if i.get_attribute("innerText") == "Temp":
        i.find_element(By.XPATH, "./p").click()
        break

Fill in the SQL code for query request.

In [111]:
i = 1
searched_rows = "100000"
time_range = ""

In [112]:
while searched_rows == "100000":

    # Create SQL code for search
    sql_code_select = "SELECT pool_name, tx_hash, block_timestamp, sender, recipient, token0_symbol, amount0_adjusted, amount0_usd, token1_symbol, amount1_adjusted, amount1_usd, price_1_0 "
    sql_code_from = "FROM ethereum.uniswapv3.ez_swaps "
    sql_code_where_pool = "WHERE pool_address = '" + pool_address + "' " 
    sql_code_where_time = "AND block_timestamp <= '" + time_range + "' " 
    sql_code_order = "ORDER BY block_timestamp DESC"

    if time_range == "":
        sql_code = sql_code_select + sql_code_from + sql_code_where_pool + sql_code_order
    else:
        sql_code = sql_code_select + sql_code_from + sql_code_where_pool + sql_code_where_time + sql_code_order

    #Fill SQL code and run query
    time.sleep(np.random.randint(2, 3))
    driver.find_element(By.XPATH, "//div[@class='cm-activeLine cm-line']").clear()
    driver.find_element(By.XPATH, "//div[@class='cm-content']").send_keys(sql_code)
    time.sleep(np.random.randint(2, 3))
    driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[2]/div[2]/div[1]/div[2]/div[1]").click()
    time.sleep(np.random.randint(2, 3))

    #Wait for query to be executed then download data
    WebDriverWait(driver, 600).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[1]/div[3]/div/div[2]/div/div/div/div[1]/div/div[2]/span[2]"))).click()
    
    #Wait for the data to be downloaded then rename it and move to according folder
    while not os.path.exists("download-query_run_results.csv"):
        time.sleep(1)
    os.rename("download-query_run_results.csv", pool_name + "_" + str(i) + ".csv")
    shutil.move(os.path.join(download_path, pool_name + "_" + str(i) + ".csv"), 
                os.path.join(download_path, pool_name, pool_name + "_" + str(i) + ".csv"))
    time.sleep(np.random.randint(5, 10))

    #See if all swap data have been download (i.e. searched rows = 100,000)
    #This will break the loop when searched rows are no longer 100,000 meaning that all data have been downloaded
    searched_rows = re.sub("[^0-9]", "", driver.find_element(By.XPATH, "//div[@class='flex mr-1']/div").get_attribute("innerText"))

    #Read lastest date in the previous download file
    time_range = pd.read_csv(os.path.join(download_path, pool_name, pool_name + "_" + str(i) + ".csv"))["BLOCK_TIMESTAMP"].dropna().iloc[-1][0:10]
    time_range = time_range + "T23:59:59Z"

    #Add 1 to i
    i = i + 1